by the way, just learned that you can put latex into markdown cells by putting it between $$

Just found out about this one, called RKF45 which stands for Runge-Kutta-Fehlberg Fourth-order-fifth-order scheme. This uses the fifth order method of

$$
\tilde{w_{i+1}}=w_i+\frac{16}{135}k_1+\frac{6656}{12825}k_3+\frac{28561}{56430}k_4-\frac{9}{50}k_5+\frac{2}{55}k_6
$$

to estimate the local truncation error in one time step of the fourth-order method

$$
w_{i+1}=w_i+\frac{25}{216}k_1+\frac{1408}{2565}k_3+\frac{2197}{4104}k_4-\frac{1}{5}k_5
$$

But what the hell does that mean? Well we defined $w_i$ as $w_i \approx y_i=y(t_i)$, so $w_i$ is the approximated $y_i$ value. Big difference between the two being that $y_i$ is the true value of our solution $y$ at time $t_i$ (or $x_i$ if we are using a different variable or whatever, its all interchangeable) and $w_i$ is our $\textit{approximation}$ of $y_i$. But what is $\tilde{w_i}$?

Alright, theres a lot that I learned from this, this is gonna take some time to digest

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [14]:
def functionInQuestion(x,t):
    return(-3*t*(x**2)+(1/(1+t**3)))

In [67]:
def rkf45(function, initialStepSize, initialPair, intervalLength, minStepSize, maxStepSize, TOL):
    totalLengthComputed = 0
    stepSize = initialStepSize
    
    y0=initialPair[1]
    x0=initialPair[0]
    
    lasty = y0
    lastx = x0
    
    solutionx = np.array([x0])
    solutiony = np.array([y0])
    
    i = 0
    while totalLengthComputed <= intervalLength:
        i += 1
        print(i)
        
        #Compute coefficients
        k1 = stepSize*function(lastx, lasty)
        k2 = stepSize*function(lastx+(1/4)*stepSize, lasty+(1/4)*k1)
        k3 = stepSize*function(lastx+(3/8)*stepSize, lasty+(3/32)*k1+(9/32)*k2)
        k4 = stepSize*function(lastx+(12/13)*stepSize, lasty+(1932/2197)*k1-(7200/2197)*k2+(7296/2197)*k3)
        k5 = stepSize*function(lastx+stepSize, lasty+(439/216)*k1-8*k2+(3680/513)*k3-(845/4104)*k4)
        k6 = stepSize*function(lastx+(1/2)*stepSize, lasty-(8/27)*k1+2*k2-(3544/2565)*k3+(1859/4104)*k4-(11/40)*k5)
        
        #Calculate actual values
        nexty = lasty+(25/216)*k1+(1408/2565)*k3+(2197/4101)*k4-(1/5)*k5
#         predictedy = lasty+(16/135)*k1+(6656/12825)*k3+(28561/56430)*k4-(9/50)*k5+(2/55)*k6
        truncatedError = ((1/360)*k1-(128/4275)*k3-(2197/75240)*k4+(1/50)*k5+(2/55)*k6)/stepSize
        q = (TOL/(2*np.absolute(truncatedError)))**(1/4)
        
        print(truncatedError)
        if truncatedError > TOL:
            stepSize = stepSize*q
            continue
        
        stepSize = stepSize*q
        if stepSize >= maxStepSize:
            stepSize = maxStepSize
        if stepSize <= minStepSize:
            stepSize = minStepSize
        else:
            stepSize = stepSize
            
        nextx = lastx+stepSize
        
        solutionx = np.append(solutionx, [nextx],axis=0)
        solutiony = np.append(solutiony, [nexty],axis=0)
        
        lasty = nexty
        lastx += stepSize
        totalLengthComputed = lastx
#         print(totalLengthComputed)
        
    solution = np.array([solutionx, solutiony])
    return(solution)